In [1]:
!pip install pennylane
!pip isntall torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 14.3 MB/s eta 0:00:00
ERROR: unknown command "isntall" - maybe you meant "install"


In [218]:
import pennylane as qml  # Importing PennyLane for quantum computing
from pennylane import numpy as np  # Importing PennyLane's NumPy for compatibility
import torch  # Importing PyTorch for potential machine learning applications
import torch.nn as nn  # Importing PyTorch's neural network module
import copy  # Importing copy module for deep copying objects
import itertools  # Importing itertools for combinatorial operations
from tqdm import tqdm  # Importing tqdm for progress tracking

class AdamOptimizer():
	"""Stochastic gradient descent optimizer with Adam
	Note: All default values are from the original Adam paper
	Parameters
	----------
	params : list, length = len(coefs_) + len(intercepts_)
		The concatenated list containing coefs_ and intercepts_ in MLP model.
		Used for initializing velocities and updating params
	learning_rate_init : float, default=0.001
		The initial learning rate used. It controls the step-size in updating
		the weights
	beta_1 : float, default=0.9
		Exponential decay rate for estimates of first moment vector, should be
		in [0, 1)
	beta_2 : float, default=0.999
		Exponential decay rate for estimates of second moment vector, should be
		in [0, 1)
	epsilon : float, default=1e-8
		Value for numerical stability
	Attributes
	----------
	learning_rate : float
		The current learning rate
	t : int
		Timestep
	ms : list, length = len(params)
		First moment vectors
	vs : list, length = len(params)
		Second moment vectors
	References
	----------
	Kingma, Diederik, and Jimmy Ba.
	"Adam: A method for stochastic optimization."
	arXiv preprint arXiv:1412.6980 (2014).
	"""

	def __init__(self, params, learning_rate_init=0.001, beta_1=0.9,
				 beta_2=0.999, epsilon=1e-8, amsgrad = False):

		self.beta_1 = beta_1
		self.beta_2 = beta_2
		self.epsilon = epsilon
		if type(learning_rate_init) == float:
			self.learning_rate_init = np.ones(len(params))*learning_rate_init
		else:
			self.learning_rate_init = np.array(learning_rate_init)
		self.t = 0
		self.ms = [np.zeros_like(param) for param in params]
		self.vs = [np.zeros_like(param) for param in params]
		self.amsgrad = amsgrad
		self.max_vs = [np.zeros_like(param) for param in params]

	def get_updates(self, grads):
		"""Get the values used to update params with given gradients
		Parameters
		----------
		grads : list, length = len(coefs_) + len(intercepts_)
			Containing gradients with respect to coefs_ and intercepts_ in MLP
			model. So length should be aligned with params
		Returns
		-------
		updates : list, length = len(grads)
			The values to add to params
		"""
		self.t += 1
		self.ms = [self.beta_1 * m + (1 - self.beta_1) * grad
					 for m, grad in zip(self.ms, grads)]
		self.vs = [self.beta_2 * v + (1 - self.beta_2) * (grad ** 2)
					 for v, grad in zip(self.vs, grads)]
		self.max_vs = [np.maximum(v, max_v) for v, max_v in zip(self.vs, self.max_vs)]
		self.learning_rate = (self.learning_rate_init *
								np.sqrt(1 - self.beta_2 ** self.t) /
								(1 - self.beta_1 ** self.t))
		if self.amsgrad:
			updates = [lr * m / (np.sqrt(max_v) + self.epsilon)
			           for lr, m, max_v in zip(self.learning_rate, self.ms, self.max_vs)]
		else:
			updates = [lr * m / (np.sqrt(v) + self.epsilon)
					 for lr, m, v in zip(self.learning_rate, self.ms, self.vs)]
		return updates



class QAOA_layer():
    def __init__(self, depth, Q):
        """
        Initializes the QAOA layer with a given depth and QUBO matrix.

        Args:
            depth (int): The depth of the QAOA circuit (number of layers).
            Q (np.ndarray): QUBO matrix representing the quadratic unconstrained binary optimization problem.
        """
        self.Q = Q  # Store the QUBO matrix
        self.p = depth  # Store the QAOA depth
        self.ham = self.prepare_cost_hamiltonian()  # Prepare the cost Hamiltonian based on QUBO matrix
        self.dev = qml.device("default.qubit", wires=Q.shape[0])  # Define a quantum device with qubits equal to the size of Q

    def qaoa_circuit(self, params):
        """
        Constructs the QAOA circuit with the given parameters.

        Args:
            params (list): A list containing gamma and beta values for parameterized QAOA layers.
        """
        n = self.Q.shape[0]  # Number of qubits based on QUBO matrix size
        gammas = params[:self.p]  # Extract gamma parameters for cost Hamiltonian evolution
        betas = params[self.p:]  # Extract beta parameters for mixer Hamiltonian
        # Apply Hadamard gates to all qubits to initialize in uniform superposition
        for i in range(n):
            qml.Hadamard(wires=i)

        # Apply QAOA layers consisting of cost and mixer Hamiltonians
        for layer in range(self.p):
            self.qubo_cost(gammas[layer])  # Apply cost Hamiltonian with corresponding gamma
            self.mixer(betas[layer])  # Apply mixer Hamiltonian with corresponding beta

    def qubo_cost(self, gamma):
        """
        Implements the cost Hamiltonian evolution for the QUBO problem.

        Args:
            gamma (float): Parameter for cost Hamiltonian evolution.
        """
        n = self.Q.shape[0]  # Get number of qubits
        for i in range(n):
            for j in range(n):
                if self.Q[i, j] != 0:  # If the QUBO coefficient is non-zero
                    if i == j:
                        qml.RZ(2 * gamma * float(self.Q[i, j]), wires=i)  # Apply single-qubit phase rotation
                    else:
                        qml.CNOT(wires=[i, j])  # Apply CNOT before controlled rotation
                        qml.RZ(2 * gamma * float(self.Q[i, j]), wires=j)  # Apply controlled phase rotation
                        qml.CNOT(wires=[i, j])  # Undo entanglement with another CNOT

    def mixer(self, beta):
        """
        Implements the mixer Hamiltonian for QAOA.

        Args:
            beta (float): Parameter for mixer Hamiltonian evolution.
        """
        for i in range(self.Q.shape[0]):
            qml.RX(2 * beta, wires=i)  # Apply X-rotation to all qubits

    def prepare_cost_hamiltonian(self):
        """
        Constructs the cost Hamiltonian for the QUBO problem.

        Returns:
            qml.Hamiltonian: The constructed cost Hamiltonian.
        """
        n = self.Q.shape[0]  # Get the size of the QUBO matrix
        coeffs = []  # Store the coefficients of the Hamiltonian terms
        ops = []  # Store the corresponding Pauli operators

        for i in range(n):
            for j in range(n):
                if self.Q[i, j] != 0:  # Only consider non-zero QUBO elements
                    if i == j:
                        coeffs.append(self.Q[i, j])  # Add coefficient for single-qubit term
                        ops.append(qml.PauliZ(i))  # Add Pauli-Z operator
                    else:
                        coeffs.append(self.Q[i, j])  # Add coefficient for two-qubit interaction
                        ops.append(qml.PauliZ(i) @ qml.PauliZ(j))  # Add ZZ interaction term

        return qml.Hamiltonian(coeffs, ops)  # Return the constructed Hamiltonian



class RL_QAOA():
    def __init__(self, Q, n_c,init_paramter,b_vector, QAOA_depth,gamma=0.99,learning_rate_init=0.001):
        self.Q = Q
        self.n_c = n_c
        self.param = init_paramter
        self.b = b_vector
        self.p = QAOA_depth
        self.qaoa_layer = QAOA_layer(QAOA_depth,Q)
        self.gamma = gamma
        self.optimzer = AdamOptimizer([init_paramter,b_vector],learning_rate_init=learning_rate_init)




    def RL_QAOA(self,episodes,epochs,correc_ans = None):
        ## RL_QAOA실행 episodes : 몬테카를로를 진행할 숫자
        ## epochs : parameter 최적화를 진행할 숫자

        for j in range(epochs):
          value_list = []
          QAOA_diff_list = []
          beta_diff_list = []
          if correc_ans is not None:
            prob = 0


          for i in tqdm(range(episodes)):
            QAOA_diff,beta_diff,value = self.rqaoa_execute()
            value_list.append(value)
            QAOA_diff_list.append(QAOA_diff)
            beta_diff_list.append(beta_diff)

            if correc_ans is not None:
              if value <= correc_ans+0.01 and value >= correc_ans-0.01:
                prob+=1


          batch_maen = np.array(value_list) - np.mean(np.array(value_list),axis=0)
          print(batch_maen)

          for index,val in enumerate(batch_maen):
            QAOA_diff_list[index] *= -val
            beta_diff_list[index] *=-val


          QAOA_diff_sum = np.mean(QAOA_diff_list,axis=0)
          beta_diff_sum = np.mean(beta_diff_list,axis=0)
          value_sum = np.mean(value_list)
          print(f'cost : {value_sum}')
          if correc_ans is not None:
              prob = prob/episodes
              print(f'prob : {prob}')
          update = self.optimzer.get_updates([QAOA_diff_sum,beta_diff_sum])
          print(f'QAOA update : {update[0]}')
          self.param += np.array(update[0])
          self.b += np.reshape(update[1],-1)
          print(f'QAOA param : {self.param}')




    def rqaoa_execute(self,cal_grad = True):
        ## RQAOA를 확률적 뽑기로 진행해 주는 함수
        Q_init = copy.deepcopy(self.Q)
        Q_action = copy.deepcopy(self.Q)
        self.Q_remove = []
        self.node_assignments = {}  # Keep track of node assignments
        self.edge_expectations = [] # Edge들의 expectation value 저장(Gradient 계산에 필요)
        self.edge_expectations_grad = [] # Edge들의 expectation value gradient 저장(Gradient 계산에 필요)
        self.policys = [] #각 edge가 지정될 확률을 저장(Gradient 계산에 필요)




        QAOA_diff_list = [] ##QAOA layer parameter gradient
        beta_diff_list = [] ##beta parameter gradient
        index = 0
        while Q_init.shape[0] > self.n_c:
            # Select and cut an edge,cal grad
            edge_expectations = self._qaoa_edge_expectations(Q_init,[i for i in range(self.p*index*2,self.p*index*2+2*self.p)])
            selected_edge_idx, policy, edge_res = self._select_edge_to_cut(Q_action,Q_init,edge_expectations)
            if cal_grad:
              #edge_res_grad = self._qaoa_edge_expectations_gradients(Q_init, [i for i in range(self.p*index*2,self.p*index*2+2*self.p)])
              edge_res_grad = self._qaoa_edge_expectations_gradient(Q_init, [i for i in range(self.p*index*2,self.p*index*2+2*self.p)],selected_edge_idx)
            if cal_grad:
              #QAOA_diff = self._compute_log_pol_diff(selected_edge_idx,Q_action,edge_res,edge_res_grad ,policy)
              QAOA_diff = self._compute_log_pol_diff_idx(selected_edge_idx,Q_action,edge_res,edge_res_grad ,policy)
              beta_diff = self._compute_grad_beta(selected_edge_idx,Q_action,policy,edge_res)


              QAOA_diff_list.append(QAOA_diff)
              beta_diff_list.append(beta_diff)
            Q_init,Q_action = self._cut_edge(selected_edge_idx,edge_res,Q_action,Q_init)
            index+=1


        self.QAOA_result = copy.deepcopy(self.node_assignments)
        # 그래프의 크기가 n_c보다 작으면 전체 space를 다 계산해서 최종값 도출
        self._brute_force_optimal(Q_init)


        z = self._dict_to_list(self.node_assignments)
        Value = self._state_energy(z,self.Q)

        ## 최근에 얻은 값이 최종 변화율에 영향을 크게 미치도록 함수 변경(큰 그래프에서는 뭘 선택하든 중요하지 않을때가 많음)


        ## 정책에 대한 미분과 panalty 받은 보상을 곱해서 최종 기울기 도출
        if cal_grad:

          beta_diff_res = None
          for beta_diff in beta_diff_list:
            if beta_diff_res is None:
              beta_diff_res = beta_diff
            else:
              beta_diff_res += beta_diff

          QAOA_diff_res = None
          for QAOA_diff in QAOA_diff_list:
            if QAOA_diff_res is None:
              QAOA_diff_res = QAOA_diff
            else:
              QAOA_diff_res += QAOA_diff

          return QAOA_diff_res,beta_diff_res,Value


        else:
          return Value




    def _compute_log_pol_diff(self, idx,Q_action,edge_expectations,edge_expectations_grad,policy):
        ## 특정 edge가 선택 됐을 때 그 정책에 대한 paramter의 log기울기 값 계산
        action_space = self._action_space(Q_action)
        betas = self.b[action_space]
        gather = np.zeros_like(policy)
        for i in range(len(edge_expectations_grad)):
            gather[i] += policy[i] * betas[i]

        diff_log_pol = betas[idx] *np.sign(edge_expectations[idx])* edge_expectations_grad[idx]
        for i in range(len(gather)):
            if gather[i]:
                diff_log_pol-= gather[i] * np.sign(edge_expectations[i]) * (
                    (edge_expectations_grad[i]))

        return np.array(diff_log_pol)

    def _compute_log_pol_diff_idx(self, idx,Q_action,edge_expectations,grad,policy):
        ## 특정 edge가 선택 됐을 때 그 정책에 대한 paramter의 log기울기 값 계산
        action_space = self._action_space(Q_action)
        betas = self.b[action_space]
        diff_log_pol = betas[idx] *np.sign(edge_expectations[idx])* grad - policy[idx] * betas[idx] *np.sign(edge_expectations[idx])* grad
        return np.array(diff_log_pol)

    def _compute_grad_beta(self,idx,Q_action,policy,edge_expectations):

        grad_beta = []
        abs_expectations = abs(np.array(edge_expectations))
        action_space = self._action_space(Q_action)

        betas_idx = action_space

        grad = np.zeros(len(self.b))
        grad[betas_idx[idx]] += abs_expectations[idx]
        for i in range(len(action_space)):
            grad[betas_idx[i]] -= policy[i] * abs_expectations[i]


        return np.array(grad)



    def _cut_edge(self, selected_edge_idx,expectations,Q_action,Q_init):
        ## 선택된 edge에 해당하는 index와 그에 해당하는 expectaion value를 받아 1 혹은 -1 를 edge의 노드에 부여하는 메소드
        ## 또한 Q_action,줄어들어진 Q도 여기서 갱신해준다
        edge_list = [(i, j) for i in range(Q_init.shape[0]) for j in range(Q_init.shape[0]) if Q_init[i, j] != 0 and i!=j]
        edge_to_cut = edge_list[selected_edge_idx]


        expectation = expectations[selected_edge_idx]
        ###수정


        i = edge_to_cut[0]
        j = edge_to_cut[1]

        for key in dict(sorted(self.node_assignments.items(), key=lambda item: item[0])):
          if i >= key:
            i+=1
          if j >= key:
            j+=1

        node_list = list(self.node_assignments.keys())
        node_list += [i,j]
        node_list.sort()


        Q_remove = []
        for index_1 in node_list:
          row = []
          for index_2 in node_list:
            row.append(self.Q[index_1][index_2])
          Q_remove.append(row)
        Q_remove = np.array(Q_remove)


        if expectation < 0:
            self.node_assignments[i] = -1
            self.node_assignments[j] = 1
            sorted_dict = np.array(list(dict(sorted(self.node_assignments.items())).values()))
            energy_1 = self._state_energy(sorted_dict,Q_remove)


            self.node_assignments[i] = 1
            self.node_assignments[j] = -1
            sorted_dict = np.array(list(dict(sorted(self.node_assignments.items())).values()))
            energy_2 = self._state_energy(sorted_dict,Q_remove)
            if energy_1 > energy_2:
              self.node_assignments[i] = 1
              self.node_assignments[j] = -1
            else:
              self.node_assignments[i] = -1
              self.node_assignments[j] = 1


        else:
            if Q_init[edge_to_cut[0]][edge_to_cut[0]]+Q_init[edge_to_cut[1]][edge_to_cut[1]]>0:
              # Assign nodes such that they are the same (e.g., 0-0 or 1-1)
              self.node_assignments[i] = -1
              self.node_assignments[j] = -1
              sorted_dict = np.array(list(dict(sorted(self.node_assignments.items())).values()))
              energy_1 = self._state_energy(sorted_dict,Q_remove)
            else:
              # Assign nodes such that they are the same (e.g., 0-0 or 1-1)
              self.node_assignments[i] = 1
              self.node_assignments[j] = 1
              sorted_dict = np.array(list(dict(sorted(self.node_assignments.items())).values()))
              energy_2 = self._state_energy(sorted_dict,Q_remove)
              if energy_1 > energy_2:
                self.node_assignments[i] = 1
                self.node_assignments[j] = 1
              else:
                self.node_assignments[i] = -1
                self.node_assignments[j] = -1

        # Remove the nodes connected by the selected edge
        nodes_to_remove = list(edge_to_cut)

        # Create a reduced Q matrix
        remaining_nodes = [i for i in range(Q_init.shape[0]) if i not in nodes_to_remove]
        new_size = len(remaining_nodes)
        new_Q = np.zeros((new_size, new_size))

        for index, node_i in enumerate(remaining_nodes):
            for jedex, node_j in enumerate(remaining_nodes):
                new_Q[index, jedex] = Q_init[node_i, node_j]


        Q_action[i] = 0
        Q_action[:,i] = 0
        Q_action[:,j] = 0
        Q_action[j] = 0



        return new_Q,Q_action

    def _select_edge_to_cut(self,Q_action,Q_init,edge_expectations):
          ## ZZ를 측정하여 beta와 곱한 후 softmax를 진행하여 자를 edge를 확률적으로 고르는 method
          action_space = self._action_space(Q_action)
          interactions = abs(np.array((edge_expectations))) * self.b[action_space]
          probabilities = torch.softmax(torch.tensor(interactions), dim=0).numpy()
          selected_edge_idx = np.random.choice(len(probabilities), p=probabilities)
          return selected_edge_idx, probabilities, edge_expectations




    def _action_space(self,Q_action):
        ## 현재 살아있는 edge들이 기존 그래프에서 몇번째 edge였는지 표시해주는 action space뽑아내주는 함수
        ## Q_action : 죽은 노드들이 있는 행과 열에 대하여 0으로 대체된 살아있는 노드들만 표시되는 매트릭스
        action_space_list = []
        index = 0
        for i in range(Q_action.shape[0]):
            for j in range(Q_action.shape[0]):
                if i!=j and self.Q[i,j] != 0:
                    if Q_action[i,j] != 0:
                        action_space_list.append(index)
                    index+=1
        return action_space_list

    def _qaoa_edge_expectations(self, Q,idx):
        ## 각 edge별 ZZ interaction을 도출해주는 함수

        @qml.qnode(self.qaoa_layer.dev)
        def circuit(param):
            self.qaoa_layer.qaoa_circuit(param)
            return [qml.expval(qml.PauliZ(i) @ qml.PauliZ(j)) for i in range(Q.shape[0]) for j in range(Q.shape[0]) if Q[i, j] != 0 and i!=j]
        return circuit(self.param[idx])

    def _qaoa_edge_expectations_gradients(self, Q,idx):
      ## 각 edge별 ZZ interaction의 기울기를 도출해주는 함수
        res = []
        @qml.qnode(self.qaoa_layer.dev)
        def circuit(params,i,j):
            self.qaoa_layer.qaoa_circuit(params[idx])
            return qml.expval(qml.PauliZ(i) @ qml.PauliZ(j))
        for i in range(Q.shape[0]) :
          for j in range(Q.shape[0]):
             if Q[i, j] != 0 and i != j:
                # Compute gradients for each parameter
                gradients = qml.grad(circuit)(self.param,i,j)
                res.append(gradients)
        return res


    def _qaoa_edge_expectations_gradient(self, Q,idx,index):
      ## 각 edge별 ZZ interaction의 기울기를 도출해주는 함수
        res = []
        number = 0
        @qml.qnode(self.qaoa_layer.dev)
        def circuit(params,i,j):
            self.qaoa_layer.qaoa_circuit(params[idx])
            return qml.expval(qml.PauliZ(i) @ qml.PauliZ(j))
        for i in range(Q.shape[0]) :
          for j in range(Q.shape[0]):
             if Q[i, j] != 0 and i != j:
                if number == index:
                  # Compute gradients for each parameter
                  gradients = qml.grad(circuit)(self.param,i,j)
                  return gradients
                number+=1


    def _brute_force_optimal(self,Q):

        ### 고전적으로 최선의 해를 도출해주는 함수
        if Q.shape[0] > self.n_c:
            print("Graph size too large for brute force.")
            return None

        n = Q.shape[0]
        configs = list(itertools.product([-1, 1], repeat=n))
        best_value = np.inf
        best_config = None



        for config in configs:
            node_assignments = {}
            copy_node = copy.deepcopy(self.node_assignments)


            for index,value in enumerate(config):
              for key in dict(sorted(self.node_assignments.items(), key=lambda item: item[0])):
                if index >= key:
                  index+=1
              node_assignments[index] = value


            copy_node.update(node_assignments)
            z = self._dict_to_list(copy_node)
            value = self._state_energy(z,self.Q)
            if value < best_value:
                best_value = value
                res_node = copy.copy(copy_node)

        self.node_assignments = res_node





    def _state_energy(self,state,Q):
        ## State가 주어질 때 그 에너지를 계산해주는 함수


        # 동일한 크기의 단위 행렬 생성
        identity_matrix = np.eye(Q.shape[0], dtype=bool)

        # 대각선 요소를 제거한 새로운 행렬 생성
        interaction = np.where(identity_matrix, 0, Q)
        diagonal_elements = np.diag(Q)
        value = diagonal_elements@state + state.T@interaction@state
        return value

    def _dict_to_list(self,node_assign):
        ##보조 함수 assign된 node들의 0 과 1을 list 형태로 순서대로 출력해주는 함수
        node_assignments_list = [0]*len(node_assign)
        for key,value in node_assign.items(): # Change: Iterate through key-value pairs using .items()
            node_assignments_list[key]=value
        return np.array(node_assignments_list)




In [140]:
def generate_upper_triangular_qubo(size, low=-10, high=10, integer=True, seed=None):
    """
    Generates an upper-triangular QUBO (Quadratic Unconstrained Binary Optimization) matrix.

    Args:
        size (int): The number of variables (size of the QUBO matrix).
        low (int/float): Minimum value of the random elements.
        high (int/float): Maximum value of the random elements.
        integer (bool): If True, generates integer values; otherwise, generates float values.
        seed (int, optional): Random seed for reproducibility.

    Returns:
        np.ndarray: An upper-triangular QUBO matrix of the specified size.
    """
    if seed is not None:
        np.random.seed(seed)

    # Generate random values for the upper triangular part including diagonal
    if integer:
        Q = np.random.randint(low, high, (size, size))
    else:
        Q = np.random.uniform(low, high, (size, size))

    # Keep only the upper triangle values (including diagonal), set lower triangle to zero
    Q = np.triu(Q)

    # Ensure diagonal values are positive (bias terms)
    np.fill_diagonal(Q, np.abs(np.diagonal(Q)))

    return Q

# Example usage
qubo_matrix = generate_upper_triangular_qubo(size=3, low=-5, high=5, integer=True, seed=70)
print("Randomly generated upper-triangular QUBO matrix:\n", qubo_matrix)

import numpy as np

# Original QUBO matrix
qubo_matrix = np.array([
    [0.0658492, -0.024853287,  0.064008761,  0.058160105,  0.058757735],
    [-0.024853287, 0.080128677, -0.025012106, -0.023698929, -0.013849719],
    [0.064008761, -0.025012106,  0.063201845,  0.053540045,  0.055350148],
    [0.058160105, -0.023698929,  0.053540045,  0.066177288,  0.055601435],
    [0.058757735, -0.013849719,  0.055350148,  0.055601435,  0.059764565]
])

diagonal = np.diag(np.diag(qubo_matrix)) + np.eye(qubo_matrix.shape[0])

# Combine results: upper triangle + modified diagonal






# Create an upper triangular matrix by doubling the upper triangle values and keeping diagonal
modified_qubo = np.triu(qubo_matrix, k=1) * 2 + np.diag(np.diag(qubo_matrix))
modified_qubo =  modified_qubo + diagonal

print("Modified QUBO Matrix:\n", modified_qubo)
qubo_matrix = modified_qubo


Randomly generated upper-triangular QUBO matrix:
 [[ 1 -3  3]
 [ 0  1  0]
 [ 0  0  1]]
Modified QUBO Matrix:
 [[ 1.1316984  -0.04970657  0.12801752  0.11632021  0.11751547]
 [ 0.          1.16025735 -0.05002421 -0.04739786 -0.02769944]
 [ 0.          0.          1.12640369  0.10708009  0.1107003 ]
 [ 0.          0.          0.          1.13235458  0.11120287]
 [ 0.          0.          0.          0.          1.11952913]]


In [212]:
import pennylane as qml
from pennylane import numpy as np
from scipy.optimize import minimize

# Define QAOA depth

''' Q = qubo_matrix
print(Q)
depth = 2 '''
Q = generate_upper_triangular_qubo(9,-5,5,integer=False)
depth = 1
# Initialize the QAOA layer
qaoa = QAOA_layer(depth, Q)

# Define the QNode for optimization
@qml.qnode(qaoa.dev)
def qaoa_expectation(params):
    """
    Quantum node that runs the QAOA circuit and measures the expectation value
    of the cost Hamiltonian.

    Args:
        params (np.ndarray): Array of QAOA parameters (gammas and betas).

    Returns:
        float: Expectation value of the cost Hamiltonian.
    """
    qaoa.qaoa_circuit(params)
    return qml.expval(qaoa.ham)

# Define the cost function for optimization
def cost_function(params):
    """
    Computes the expectation value of the cost Hamiltonian.

    Args:
        params (np.ndarray): Array of QAOA parameters (gammas and betas).

    Returns:
        float: Expectation value of the cost Hamiltonian.
    """
    return qaoa_expectation(params)







opt = qml.AdamOptimizer(stepsize=0.1)

# Initial random parameters for QAOA (gammas and betas)
init_params = np.random.uniform(0, np.pi, 2 * depth)

# Define the QNode for optimization
@qml.qnode(qaoa.dev)
def qaoa_expectation(params):
    """
    Quantum node that runs the QAOA circuit and measures the expectation value
    of the cost Hamiltonian.

    Args:
        params (np.ndarray): Array of QAOA parameters (gammas and betas).

    Returns:
        float: Expectation value of the cost Hamiltonian.
    """
    qaoa.qaoa_circuit(params)
    return qml.expval(qaoa.ham)

# Define the cost function for optimization
def cost_function(params):
    """
    Computes the expectation value of the cost Hamiltonian.

    Args:
        params (np.ndarray): Array of QAOA parameters (gammas and betas).

    Returns:
        float: Expectation value of the cost Hamiltonian.
    """
    return qaoa_expectation(params)

# Initial random parameters for QAOA (gammas and betas)
init_params = np.random.uniform(0, np.pi, 2 * depth)

# Optimize the QAOA parameters using classical optimizer (COBYLA)
result = minimize(cost_function, init_params, method="COBYLA", options={'maxiter': 200})

# Extract optimized parameters and minimum cost
optimized_params = result.x
min_cost = result.fun

# Print the results
print("Optimized parameters (gammas and betas):", optimized_params)
print("Minimum cost achieved:", min_cost)


Optimized parameters (gammas and betas): [3.62673162 4.2089466 ]
Minimum cost achieved: -2.243267672750182


In [213]:
import pennylane as qml
from pennylane import numpy as np

# Define the QUBO matrix for the problem
def generate_upper_triangular_qubo(size, low=-10, high=10, integer=True, seed=None):
    """
    Generates an upper-triangular QUBO (Quadratic Unconstrained Binary Optimization) matrix.

    Args:
        size (int): The number of variables (size of the QUBO matrix).
        low (int/float): Minimum value of the random elements.
        high (int/float): Maximum value of the random elements.
        integer (bool): If True, generates integer values; otherwise, generates float values.
        seed (int, optional): Random seed for reproducibility.

    Returns:
        np.ndarray: An upper-triangular QUBO matrix of the specified size.
    """
    if seed is not None:
        np.random.seed(seed)

    # Generate random values for the upper triangular part including diagonal
    if integer:
        Q = np.random.randint(low, high, (size, size))
    else:
        Q = np.random.uniform(low, high, (size, size))

    # Keep only the upper triangle values (including diagonal), set lower triangle to zero
    Q = np.triu(Q)

    # Ensure diagonal values are positive (bias terms)
    np.fill_diagonal(Q, np.abs(np.diagonal(Q)))

    return Q



# Initialize the QAOA layer
qaoa = QAOA_layer(depth, Q)

# Define the QNode for optimization
@qml.qnode(qaoa.dev)
def qaoa_expectation(params):
    """
    Quantum node that runs the QAOA circuit and measures the expectation value
    of the cost Hamiltonian.

    Args:
        params (np.ndarray): Array of QAOA parameters (gammas and betas).

    Returns:
        float: Expectation value of the cost Hamiltonian.
    """
    qaoa.qaoa_circuit(params)
    return qml.expval(qaoa.ham)

# Define the cost function for optimization
def cost_function(params):
    """
    Computes the expectation value of the cost Hamiltonian.

    Args:
        params (np.ndarray): Array of QAOA parameters (gammas and betas).

    Returns:
        float: Expectation value of the cost Hamiltonian.
    """
    return qaoa_expectation(params)







opt = qml.AdamOptimizer(stepsize=0.1)

# Initial random parameters for QAOA (gammas and betas)
init_params = optimized_params

# Optimization loop
max_iterations = 200
params = init_params
for i in range(max_iterations):
    params = opt.step(cost_function, params)
    if i % 20 == 0:
        cost = cost_function(params)
        print(f"Iteration {i}: Cost = {cost}")

# Final optimized parameters
final_cost = cost_function(params)
print("Optimized parameters (gammas and betas):", params)
print("Final minimum cost achieved:", final_cost)





Iteration 0: Cost = -2.243267672750182
Iteration 20: Cost = -2.243267672750182
Iteration 40: Cost = -2.243267672750182
Iteration 60: Cost = -2.243267672750182
Iteration 80: Cost = -2.243267672750182
Iteration 100: Cost = -2.243267672750182
Iteration 120: Cost = -2.243267672750182
Iteration 140: Cost = -2.243267672750182
Iteration 160: Cost = -2.243267672750182
Iteration 180: Cost = -2.243267672750182
Optimized parameters (gammas and betas): [3.62673162 4.2089466 ]
Final minimum cost achieved: -2.243267672750182


In [214]:
dev = qml.device("default.qubit", shots=1500)
@qml.qnode(dev)
def sample_solution(params):
    """Runs the optimized QAOA circuit and samples the output state."""
    qaoa.qaoa_circuit(params)
    return qml.sample()

# Get multiple samples from the optimized circuit
num_samples = 1000
samples = sample_solution(params)

# Convert measurement results from {-1,1} to {0,1}
binary_solutions = samples*(-2)+1

# Compute the cost for each sampled solution
def evaluate_solution(sample):
    # 동일한 크기의 단위 행렬 생성
    identity_matrix = np.eye(Q.shape[0], dtype=bool)

    # 대각선 요소를 제거한 새로운 행렬 생성
    interaction = np.where(identity_matrix, 0, Q)
    diagonal_elements = np.diag(Q)
    value = diagonal_elements@sample + sample.T@interaction@sample
    return value

# Evaluate the cost for all sampled solutions and find the best
costs = np.array([evaluate_solution(sample) for sample in binary_solutions])
best_solution = binary_solutions[np.argmin(costs)]
best_cost = np.min(costs)

print("Best solution found:", best_solution)
print("Cost of best solution:", best_cost)

Best solution found: [ 1  1 -1 -1 -1 -1 -1  1  1]
Cost of best solution: -50.13978872011384


In [215]:
@qml.qnode(dev)
def sample_solution(params):
    """Runs the optimized QAOA circuit and samples the output state."""
    qaoa.qaoa_circuit(params)
    return qml.sample()

In [216]:
# Initial parameters for QAOA
init_params = params
init_params = np.reshape(init_params,(-1))

# RL-QAOA setup
rl_qaoa = RL_QAOA(Q,Q.shape[0],init_params,b_vector = np.array([25.]*30) ,QAOA_depth=1,gamma = 0.99,learning_rate_init=[0.01,0.05])
final_config = rl_qaoa.rqaoa_execute()
rl_qaoa.n_c = 1
print(f"classical_result : {final_config},best : {rl_qaoa.node_assignments}" )

classical_result : (None, None, tensor(-50.13978872, requires_grad=True)),best : {0: 1, 1: 1, 2: -1, 3: -1, 4: -1, 5: -1, 6: -1, 7: 1, 8: 1}


In [217]:
# Evaluate the cost for all sampled solutions and find the best
costs = np.array([evaluate_solution(sample) for sample in binary_solutions])
best_solution = binary_solutions[np.argmin(costs)]
best_cost = np.min(costs)

print("Best solution found:", best_solution)
print("Cost of best solution:", best_cost)
print("Best solution percent : ",np.sum(np.array((costs == float(final_config[2])),dtype=int))/1500)

Best solution found: [ 1  1 -1 -1 -1 -1 -1  1  1]
Cost of best solution: -50.13978872011384
Best solution percent :  0.0013333333333333333


In [221]:
### Single optimization
# Example QUBO matrix
#Q = Q_onsite

# Initial parameters for QAOA
init_params = [params]*5
init_params = np.reshape(init_params,(-1))

# RL-QAOA setup
rl_qaoa = RL_QAOA(Q,Q.shape[0],init_params,b_vector = np.array([15.]*70) ,QAOA_depth=depth,gamma = 0.99,learning_rate_init=[0.05,0.00])
final_config = rl_qaoa.rqaoa_execute()
rl_qaoa.n_c = 1
print(f"classical_result : {final_config},best : {rl_qaoa.node_assignments}" )


# Execute RQAOA
final_config = rl_qaoa.RL_QAOA(episodes=50,epochs=60,correc_ans=float(final_config[2]))

classical_result : (None, None, tensor(-50.13978872, requires_grad=True)),best : {0: 1, 1: 1, 2: -1, 3: -1, 4: -1, 5: -1, 6: -1, 7: 1, 8: 1}


100%|██████████| 50/50 [00:43<00:00,  1.15it/s]


[ 23.87519773 -17.01636045  -2.81128167  24.4578509   -5.54662416
  -7.80361635   7.06632909 -17.01636045  -7.80361635   0.21551871
 -20.98353724 -21.45639083 -17.02575082  12.25160525  -2.19704903
 -23.04131042 -17.01636045  46.59014461 -11.66286189  27.46540939
 -15.73969434  -3.87761191 -16.0740248   -0.99897074  14.46010766
 -11.02770154   5.6719305   -2.81128167  23.87519773  18.98924923
   7.83537292   2.63557655   2.01310057   5.54242602  28.65616642
   3.06211681  -2.01151514  27.46540939   2.77320976   2.71503876
 -15.73969434   1.85972284   0.46107569  -8.73835015  -2.19704903
   2.03665081   5.6719305  -21.45639083 -17.78931689  -7.80361635]
cost : -27.098478297257056
prob : 0.02
QAOA update : [ 0.05        0.04999993  0.05        0.05        0.05       -0.04999998
  0.05       -0.04999997  0.          0.        ]
QAOA param : [3.67673162 4.25894653 3.67673162 4.2589466  3.67673162 4.15894662
 3.67673162 4.15894663 3.62673162 4.2089466 ]


100%|██████████| 50/50 [00:39<00:00,  1.27it/s]


[ -9.10242117  -0.08623478 -21.11603006   7.59721086  17.9534896
  -6.81306979 -12.81402297  25.90343973  -3.6213438    5.01583092
  10.22394071 -12.81402297   8.99160946  -2.91139825   2.14079908
  -6.81306979   8.99160946  -1.33247322 -12.81402297  10.22394071
   9.08193517 -15.09108008  -5.87833598 -13.81441397  -1.80936308
   0.92630962  25.55883217  -3.99263038  -0.92247096 -19.05825687
  -3.6213438    4.61349601  -3.6213438   22.9408254    1.88707959
  13.45010264  22.9408254  -15.86403652  -3.68002697 -19.05825687
  -1.1486845   -4.79494403  17.9006742   13.45010264   9.19303629
 -14.14874444   4.61349601 -21.11603006   8.40823083 -14.14874444]
cost : -29.023758662174703
prob : 0.04
QAOA update : [-0.03502635 -0.03536501  0.02879792  0.03492822  0.02982825 -0.04008367
  0.03383043 -0.01718584  0.          0.        ]
QAOA param : [3.64170526 4.22358152 3.70552954 4.29387482 3.70655987 4.11886295
 3.71056205 4.14176079 3.62673162 4.2089466 ]


...

[  1.02666622   9.22328174 -16.0314156   -4.96232311   9.22328174
  -3.07877732 -14.31612352  12.88704544   0.75893054   2.33202099
  -3.78872288   8.82423038   0.74857417  -4.96232311  -9.9049606
   7.42983178   9.92413041 -19.22563595   2.33202099  -1.05338039
  -1.31606358  -9.26980025   1.97342     15.06686296  35.49198534
 -19.22563595  22.77344632  -1.34101026  17.78611052 -13.98179305
  -6.04571506  17.69331702   1.71970051  -6.04571506 -19.22563595
 -16.0314156    1.69002319  24.71992341  -4.96232311 -21.28340914
  -9.26980025   1.97342     -3.84740605  22.56232283   8.82423038
   0.59446338  -6.04571506 -21.28340914  -9.9049606    8.82423038]
cost : -28.85637958236449
prob : 0.04
QAOA update : [0.00961482 0.02077373 0.01029047 0.01863098 0.01375515 0.00461256
 0.00196489 0.00654907 0.         0.        ]
QAOA param : [3.56328284 4.39682369 3.65845511 4.64247108 3.64679981 3.84365879
 3.68875681 3.96223836 3.62673162 4.2089466 ]


100%|██████████| 50/50 [00:37<00:00,  1.32it/s]


[ 24.01405612   7.62288167   0.0689405   12.1614463   -0.11659338
  -6.6578945    7.62288167   0.0689405   -0.73082602 -12.47836367
 -13.99356915 -15.70886124 -12.47836367 -15.70886124  -0.99350922
  -6.6578945    9.14678474  -8.94724589   4.79549441  -3.46616851
  11.34427069  -5.7231607   12.29438032   9.78025777   1.07112853
   9.14678474  -6.42948528  16.81371003   9.14678474  -6.42948528
  -2.75622296   4.79549441  20.71453155 -13.99356915   1.34922059
  -5.60217858  20.71453155  -8.94724589  -0.99350922   1.08148491
 -14.94529517  22.88487719  -5.60217858   2.29597436 -13.99356915
   0.0689405   -0.73082602  -6.6578945   -5.60217858 -12.65884769]
cost : -29.178933945449554
prob : 0.0
QAOA update : [1.13526370e-02 2.02067124e-02 1.02270399e-02 1.57161868e-02
 1.56407570e-02 4.79622543e-03 1.45686424e-05 4.14944260e-03
 0.00000000e+00 0.00000000e+00]
QAOA param : [3.57463548 4.4170304  3.66868215 4.65818726 3.66244057 3.84845502
 3.68877137 3.9663878  3.62673162 4.2089466 ]


 10%|█         | 5/50 [00:04<00:43,  1.03it/s]


KeyboardInterrupt: 

In [209]:

# Example QUBO matrix
#Q = Q_onsite

# Initial parameters for QAOA
init_params = [params]*3
init_params = np.reshape(init_params,(-1))

# RL-QAOA setup
rl_qaoa = RL_QAOA(Q,Q.shape[0],init_params,b_vector = np.array([15.]*30) ,QAOA_depth=depth,gamma = 0.99,learning_rate_init=[0.05,0.00])
final_config = rl_qaoa.rqaoa_execute()
rl_qaoa.n_c = 1
print(f"classical_result : {final_config},best : {rl_qaoa.node_assignments}" )


# Execute RQAOA
final_config = rl_qaoa.RL_QAOA(episodes=50,epochs=60,correc_ans=float(final_config[2]))

classical_result : (None, None, tensor(-36.72102804, requires_grad=True)),best : {0: -1, 1: -1, 2: -1, 3: -1, 4: -1}


100%|██████████| 50/50 [00:10<00:00,  4.90it/s]


[-3.92313542 -3.92313542 15.44386196 -3.92313542 -3.92313542 15.44386196
 -3.92313542 -3.92313542 -3.92313542 -3.92313542 -3.92313542 15.44386196
 -3.92313542 -3.92313542 -3.92313542 -3.92313542 -3.92313542 -3.92313542
 -3.92313542 -3.92313542 -3.92313542 -3.92313542 -3.92313542 -3.92313542
 -3.92313542 -3.92313542 -3.92313542 15.44386196 -3.92313542 -3.92313542
 15.44386196 15.44386196 -3.92313542 -3.92313542 -3.92313542 15.44386196
 -3.92313542 -3.92313542 15.44386196 -3.92313542 -3.92313542 -3.92313542
 -3.92313542 -3.92313542 17.93065922 -3.92313542 -3.92313542 15.44386196
 -3.92313542 -3.92313542]
cost : -32.79789261385608
prob : 0.8
QAOA update : [-0.04999999 -0.04999965  0.05       -0.05        0.          0.        ]
QAOA param : [3.33854764 2.23756881 3.43854762 2.23756846 3.38854762 2.28756846]


100%|██████████| 50/50 [00:06<00:00,  7.42it/s]


[  7.36778986   4.8809926  -14.48600478 -14.48600478  19.81148398
   7.36778986   7.36778986   9.42242958   7.36778986   4.71371273
 -14.48600478   7.36778986 -14.48600478   7.36778986 -14.48600478
 -14.48600478   4.71371273   4.71371273   7.36778986 -14.48600478
   7.36778986   4.71371273   7.36778986   7.36778986   7.36778986
 -14.48600478   7.36778986 -14.48600478 -14.48600478   9.42242958
 -14.48600478 -14.48600478   7.36778986   7.36778986   4.71371273
   7.36778986 -14.48600478   7.36778986   4.8809926  -14.48600478
   4.71371273  19.81148398   4.8809926  -14.48600478   4.8809926
   7.36778986 -14.48600478   7.36778986 -14.48600478   7.36778986]
cost : -22.23502325596314
prob : 0.34
QAOA update : [ 0.03303596 -0.0372503  -0.00273512 -0.05005955  0.          0.        ]
QAOA param : [3.3715836  2.20031852 3.43581251 2.18750891 3.38854762 2.28756846]


...

[-0.38733995 -0.38733995 -0.38733995 -0.38733995 -0.38733995 -0.38733995
 -0.38733995 -0.38733995 -0.38733995 -0.38733995 -0.38733995 -0.38733995
 -0.38733995 -0.38733995 18.97965743 -0.38733995 -0.38733995 -0.38733995
 -0.38733995 -0.38733995 -0.38733995 -0.38733995 -0.38733995 -0.38733995
 -0.38733995 -0.38733995 -0.38733995 -0.38733995 -0.38733995 -0.38733995
 -0.38733995 -0.38733995 -0.38733995 -0.38733995 -0.38733995 -0.38733995
 -0.38733995 -0.38733995 -0.38733995 -0.38733995 -0.38733995 -0.38733995
 -0.38733995 -0.38733995 -0.38733995 -0.38733995 -0.38733995 -0.38733995
 -0.38733995 -0.38733995]
cost : -36.33368808783086
prob : 0.98
QAOA update : [ 0.0247898  -0.02168533 -0.01393093 -0.0338337   0.          0.        ]
QAOA param : [3.41149061 2.12263576 3.3824982  2.06272548 3.38854762 2.28756846]


100%|██████████| 50/50 [00:07<00:00,  6.78it/s]


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
cost : -36.721028035474816
prob : 1.0
QAOA update : [ 0.0213656  -0.01868995 -0.01200666 -0.02916027  0.          0.        ]
QAOA param : [3.43285621 2.10394581 3.37049154 2.03356521 3.38854762 2.28756846]


 28%|██▊       | 14/50 [00:02<00:05,  6.88it/s]


KeyboardInterrupt: 

In [211]:
### Single optimization
# Example QUBO matrix
#Q = Q_onsite

# Initial parameters for QAOA
init_params = [params]*3
init_params = np.reshape(init_params,(-1))

# RL-QAOA setup
rl_qaoa = RL_QAOA(Q,Q.shape[0],init_params,b_vector = np.array([5.]*30) ,QAOA_depth=depth,gamma = 0.99,learning_rate_init=[0.05,0.00])
final_config = rl_qaoa.rqaoa_execute()
rl_qaoa.n_c = 1
print(f"classical_result : {final_config},best : {rl_qaoa.node_assignments}" )


# Execute RQAOA
final_config = rl_qaoa.RL_QAOA(episodes=50,epochs=60,correc_ans=float(final_config[2]))

classical_result : (None, None, tensor(-36.72102804, requires_grad=True)),best : {0: -1, 1: -1, 2: -1, 3: -1, 4: -1}


100%|██████████| 50/50 [00:29<00:00,  1.72it/s]


[-10.49508838 -10.49508838 -10.49508838  11.35870626   8.871909
 -10.49508838 -10.49508838 -10.49508838 -10.49508838 -10.49508838
 -10.49508838 -10.49508838   8.871909   -10.49508838 -10.49508838
   8.871909     8.871909   -10.49508838  30.6347972   23.34627595
  23.34627595  13.41334598 -10.49508838  11.35870626 -10.49508838
  23.34627595 -10.49508838  11.35870626  23.34627595 -10.49508838
 -10.49508838 -10.49508838  11.35870626 -10.49508838  13.41334598
 -10.49508838 -10.49508838 -10.49508838  23.34627595 -10.49508838
  30.6347972  -10.49508838   8.871909    11.35870626 -10.49508838
 -10.49508838 -10.49508838 -10.49508838 -10.49508838   8.871909  ]
cost : -26.225939655278612
prob : 0.6
QAOA update : [ 0.05 -0.05  0.05 -0.05  0.    0.  ]
QAOA param : [3.43854762 2.23756846 3.43854762 2.23756846 3.38854762 2.28756846]


100%|██████████| 50/50 [00:26<00:00,  1.88it/s]


[-16.09406339   7.81437097  15.23842038   7.81437097   3.10565411
   3.27293399 -16.09406339   3.27293399   3.27293399   7.81437097
   3.10565411 -16.09406339 -16.09406339   3.10565411   7.81437097
   3.27293399   7.81437097   5.75973125   3.10565411 -16.09406339
   3.10565411 -16.09406339 -16.09406339   5.75973125   3.27293399
   7.81437097  17.74730093  15.23842038  17.99428723 -16.09406339
   7.81437097  25.03582219   3.27293399   3.27293399   3.10565411
   3.27293399   7.81437097 -16.09406339   3.10565411 -16.09406339
 -16.09406339 -16.09406339   3.10565411 -16.09406339   5.75973125
 -16.09406339   5.75973125   7.81437097 -16.09406339   5.75973125]
cost : -20.626964642677763
prob : 0.3
QAOA update : [-0.00976394 -0.04534998  0.01133634 -0.05002642  0.          0.        ]
QAOA param : [3.42878368 2.19221848 3.44988397 2.18754204 3.38854762 2.28756846]


...

[ 9.45544484 12.10952198 -9.74427266 -9.74427266 -9.74427266 -9.74427266
  9.62272472  9.62272472 14.25353196  9.45544484 -9.74427266 -9.74427266
  9.45544484 12.10952198 -9.74427266 -9.74427266  9.45544484 -9.74427266
 -9.74427266 31.38561291 -9.74427266  9.45544484  9.45544484  9.62272472
  9.45544484 -9.74427266 -9.74427266 12.10952198 -9.74427266 -9.74427266
 31.38561291 -9.74427266  9.62272472 14.1641617  12.10952198 -9.74427266
 -9.74427266  9.45544484 -9.74427266 -9.74427266 -9.74427266  9.62272472
 -9.74427266 -9.74427266  9.45544484 -9.74427266 -9.74427266 -9.74427266
 -9.74427266 -9.74427266]
cost : -26.97675537208077
prob : 0.56
QAOA update : [-0.01356604 -0.01821045 -0.02095697 -0.04868234  0.          0.        ]
QAOA param : [3.38619481 2.09080699 3.41151986 2.03945637 3.38854762 2.28756846]


100%|██████████| 50/50 [00:27<00:00,  1.85it/s]


[-2.37377563 -2.37377563 -2.37377563 -2.37377563 16.99322175 -2.37377563
 -2.37377563 -2.37377563 -2.37377563 -2.37377563 -2.37377563 -2.37377563
 -2.37377563 -2.37377563 -2.37377563 -2.37377563 -2.37377563 -2.37377563
 -2.37377563 -2.37377563 -2.37377563 -2.37377563 16.99322175 -2.37377563
 -2.37377563 -2.37377563 -2.37377563 19.48001901 -2.37377563 -2.37377563
 -2.37377563 -2.37377563 -2.37377563 -2.37377563 -2.37377563 16.99322175
 -2.37377563 -2.37377563 -2.37377563 16.99322175 -2.37377563 16.99322175
 -2.37377563 -2.37377563 -2.37377563 -2.37377563 -2.37377563 -2.37377563
 -2.37377563 -2.37377563]
cost : -34.34725240443193
prob : 0.88
QAOA update : [-0.00622876 -0.01558357 -0.01732001 -0.04156314  0.          0.        ]
QAOA param : [3.37996604 2.07522342 3.39419985 1.99789323 3.38854762 2.28756846]


 18%|█▊        | 9/50 [00:05<00:26,  1.55it/s]


KeyboardInterrupt: 

In [198]:
### Single optimization
# Example QUBO matrix
#Q = Q_onsite

# Initial parameters for QAOA
init_params = [params]*3
init_params = np.reshape(init_params,(-1))

# RL-QAOA setup
rl_qaoa = RL_QAOA(Q,Q.shape[0],init_params,b_vector = np.array([5.]*30) ,QAOA_depth=depth,gamma = 0.99,learning_rate_init=[0.05,0.00])
final_config = rl_qaoa.rqaoa_execute()
rl_qaoa.n_c = 1
print(f"classical_result : {final_config},best : {rl_qaoa.node_assignments}" )


# Execute RQAOA
final_config = rl_qaoa.RL_QAOA(episodes=50,epochs=60,correc_ans=float(final_config[2]))

classical_result : (None, None, tensor(-21.43789003, requires_grad=True)),best : {0: -1, 1: 1, 2: 1, 3: -1, 4: -1}


100%|██████████| 50/50 [00:12<00:00,  3.86it/s]


[-0.35545076 -0.35545076 -0.11368019 -0.35545076 -0.35545076 -0.35545076
 -0.35545076 -0.35545076 -0.35545076 -0.35545076 -0.35545076  4.03163853
 -0.35545076 -0.11368019  4.03163853 -5.71841722 -0.35545076 -0.35545076
  4.03163853 -0.35545076 -0.35545076 -0.35545076 -0.35545076 -0.35545076
 -0.35545076 -0.35545076 -0.11368019 -0.35545076 -0.35545076 -0.35545076
 -0.35545076 -0.35545076 -0.35545076 -0.11368019 -0.35545076 -0.11368019
 -0.11368019 -0.35545076 -0.11368019 -0.35545076 -0.35545076 -0.11368019
 -0.11368019 -5.71841722  4.03163853 -0.35545076 -0.35545076  4.03163853
 -0.35545076  4.03163853]
cost : -15.719472807294398
prob : 0.04
QAOA update : [ 0.04999995  0.05        0.04999999  0.04999999  0.04999997 -0.04999973
  0.04999994 -0.04999994  0.          0.          0.          0.        ]
QAOA param : [1.7184651  3.30284643 2.32502639 2.77377668 1.71846513 3.2028467
 2.32502634 2.67377675 1.66846515 3.25284643 2.27502641 2.72377669]


100%|██████████| 50/50 [00:12<00:00,  4.03it/s]


[ 5.96362359  0.0704135  -1.76328882 -2.00505939  5.96362359 -0.74109053
  5.96362359  5.96362359 -1.76328882 -2.00505939 -2.00505939 -2.00505939
 -1.76328882  2.3820299   0.60716745 -0.74109053 -1.76328882 -2.00505939
  0.0704135  -1.76328882 -2.00505939 -2.00505939 -1.76328882 -2.00505939
  0.60716745 -1.76328882  2.3820299   0.0704135  -0.74109053 -2.00505939
  2.3820299  -2.00505939 -2.00505939  2.3820299   0.0704135   2.3820299
  5.96362359 -2.00505939 -1.76328882 -1.76328882 -1.76328882 -1.76328882
 -2.00505939  0.0704135   5.96362359 -1.76328882 -1.76328882  5.96362359
 -2.00505939 -2.00505939]
cost : -14.06986417848556
prob : 0.0
QAOA update : [-0.03665359 -0.01811952  0.04747598  0.03346484  0.04193755 -0.04237073
  0.04184512 -0.03993826  0.          0.          0.          0.        ]
QAOA param : [1.68181151 3.28472691 2.37250238 2.80724152 1.76040268 3.16047598
 2.36687147 2.63383849 1.66846515 3.25284643 2.27502641 2.72377669]


...

[-4.68604923  5.06400652  0.67691723  0.9186878   0.9186878   0.67691723
  0.9186878   0.9186878  -4.68604923  5.06400652  1.94088609 -4.68604923
  0.67691723  0.9186878   0.9186878   0.67691723 -4.68604923  0.67691723
  0.9186878   5.06400652 -4.68604923  0.9186878   3.28914407  0.9186878
 -4.68604923  0.67691723  0.67691723  0.9186878   0.67691723  0.67691723
  0.9186878  -4.68604923  5.06400652  0.9186878   0.9186878  -4.68604923
  0.67691723 -4.68604923 -4.68604923  4.2574216   0.9186878   0.67691723
  0.67691723 -4.68604923  0.9186878   3.28914407 -4.68604923  0.9186878
 -4.68604923  5.06400652]
cost : -16.75184079817584
prob : 0.26
QAOA update : [-0.00104452 -0.01193548  0.01418107  0.0105218   0.0014423  -0.02257686
 -0.01470684 -0.01811365  0.          0.          0.          0.        ]
QAOA param : [1.6694336  3.34556232 2.6558478  2.84348492 1.58780227 3.01959105
 2.3367246  2.41321355 1.66846515 3.25284643 2.27502641 2.72377669]


100%|██████████| 50/50 [00:12<00:00,  4.02it/s]


[ 1.270218    1.270218    4.60895179  1.270218   -4.33451904  1.02844743
  1.270218   -4.33451904  3.64067427  8.99713041  1.270218    1.02844743
 -4.33451904  2.29241628  1.02844743  5.41553672 -4.33451904 -4.33451904
  3.64067427  1.270218   13.33717944  5.41553672 -4.33451904 -4.33451904
  8.99713041  1.02844743 -4.33451904 -4.33451904  1.270218    1.02844743
 -4.33451904  1.02844743  1.270218    1.270218   -4.33451904 -4.33451904
  1.270218   -4.33451904 -4.33451904  1.02844743  1.270218   -4.33451904
  1.270218   -4.33451904  1.270218   -4.33451904  1.02844743 -4.33451904
  1.270218   -4.33451904]
cost : -17.103370994445054
prob : 0.38
QAOA update : [-0.01165234 -0.02218694  0.00422723  0.00688712 -0.00823733 -0.0108689
 -0.00605283 -0.01663642  0.          0.          0.          0.        ]
QAOA param : [1.65778126 3.32337538 2.66007502 2.85037204 1.57956494 3.00872215
 2.33067177 2.39657714 1.66846515 3.25284643 2.27502641 2.72377669]


 44%|████▍     | 22/50 [00:04<00:05,  4.86it/s]Exception ignored in: <function _xla_gc_callback at 0x798038e472e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 
 72%|███████▏  | 36/50 [00:08<00:03,  4.33it/s]


KeyboardInterrupt: 

In [193]:
### Single optimization
# Example QUBO matrix
#Q = Q_onsite

# Initial parameters for QAOA
init_params = [params]*3
init_params = np.reshape(init_params,(-1))

# RL-QAOA setup
rl_qaoa = RL_QAOA(Q,Q.shape[0],init_params,b_vector = np.array([5.]*30) ,QAOA_depth=depth,gamma = 0.99,learning_rate_init=[0.05,0.00])
final_config = rl_qaoa.rqaoa_execute()
rl_qaoa.n_c = 1
print(f"classical_result : {final_config},best : {rl_qaoa.node_assignments}" )


# Execute RQAOA
final_config = rl_qaoa.RL_QAOA(episodes=50,epochs=60,correc_ans=float(final_config[2]))

classical_result : (None, None, tensor(-21.43789003, requires_grad=True)),best : {0: -1, 1: 1, 2: 1, 3: -1, 4: -1}


100%|██████████| 50/50 [00:17<00:00,  2.88it/s]


[-0.92093656 -0.92093656 -0.92093656 -0.92093656 -0.92093656 -0.92093656
 -0.92093656 -0.92093656 -0.92093656 -0.92093656 -0.92093656 -0.92093656
 -0.67916599 -0.92093656 -0.67916599 -0.92093656  3.46615273 -0.92093656
 -0.92093656  3.46615273  3.46615273  3.46615273  3.46615273 -0.92093656
 -0.92093656 -0.92093656 -0.92093656 -0.67916599 -0.67916599 -0.67916599
 -0.67916599 -0.92093656 -0.92093656 -0.92093656  3.46615273  3.46615273
 -0.67916599  3.46615273 -0.92093656 -0.92093656 -0.92093656  3.46615273
  3.46615273 -0.92093656 -0.67916599 -0.67916599 -0.92093656 -0.92093656
 -0.92093656 -0.92093656]
cost : -15.153987005760824
prob : 0.0
QAOA update : [ 0.04999997  0.05        0.04999999  0.04999998  0.04999999 -0.05
  0.04999996 -0.04999998  0.          0.          0.          0.        ]
QAOA param : [1.71846512 3.30284643 2.32502639 2.77377667 1.71846514 3.20284643
 2.32502636 2.67377671 1.66846515 3.25284643 2.27502641 2.72377669]


100%|██████████| 50/50 [00:13<00:00,  3.75it/s]


[-1.05035127  3.09496745  0.78335105 -1.05035127  1.320105   -1.29212184
 -1.29212184  1.320105   -1.29212184  1.320105    6.67656114  6.67656114
  1.320105   -1.29212184 -1.05035127 -1.05035127 -1.29212184  0.78335105
 -1.05035127  6.67656114 -1.29212184 -1.05035127 -1.05035127 -0.02815299
  6.67656114 -1.29212184  1.320105   -1.29212184 -1.29212184 -1.29212184
 -1.29212184 -1.05035127  0.78335105 -1.05035127 -1.29212184 -1.29212184
  3.09496745 -0.02815299 -1.05035127 -1.29212184  0.78335105 -1.05035127
 -1.29212184 -1.29212184 -1.29212184 -6.65508831 -0.02815299 -1.29212184
 -0.02815299 -1.05035127]
cost : -14.78280172529517
prob : 0.02
QAOA update : [-0.03615128  0.00111376  0.04925101  0.04578382  0.03267131 -0.03332896
  0.03046543 -0.03315372  0.          0.          0.          0.        ]
QAOA param : [1.68231384 3.30396019 2.37427741 2.81956049 1.75113645 3.16951748
 2.3554918  2.64062299 1.66846515 3.25284643 2.27502641 2.72377669]


...

[-2.55263862 -2.55263862 -2.55263862  4.88580073 -2.55263862 -2.55263862
 -2.55263862  5.42255468 -2.55263862 -2.55263862  3.05209841 -2.55263862
 -2.55263862 12.0486513  -2.55263862 -2.55263862  2.81032784 -2.55263862
 -2.55263862 -2.55263862 -2.55263862  4.07429669  7.19741713 -2.55263862
  3.05209841  3.05209841 -2.55263862  2.81032784 -2.55263862  5.42255468
 -2.55263862 -2.55263862 -2.55263862  4.07429669 -2.55263862 -2.55263862
 -2.55263862  4.88580073  4.07429669 -2.55263862 -2.55263862 -2.55263862
 -2.55263862  2.81032784  6.39083221 -2.55263862 -2.55263862  2.81032784
 -2.55263862  2.81032784]
cost : -18.885251406501784
prob : 0.64
QAOA update : [-0.00531367 -0.00650508  0.0072992   0.00812106 -0.00306393 -0.00776106
 -0.00053047 -0.00799706  0.          0.          0.          0.        ]
QAOA param : [1.63714309 3.32963336 2.59450644 2.84575078 1.59197944 3.1174
 2.4841099  2.64041256 1.66846515 3.25284643 2.27502641 2.72377669]


100%|██████████| 50/50 [00:12<00:00,  3.93it/s]


[-1.91030745 -1.91030745  6.06488586 -1.91030745 -1.91030745 -1.91030745
 -1.91030745 -1.91030745 -1.91030745 -1.91030745 -1.91030745 -1.91030745
 -1.91030745 -1.91030745  3.69442958  3.69442958  7.8397483  -1.91030745
  3.45265902 -1.91030745 -1.91030745  3.45265902 -1.91030745 -1.91030745
  3.45265902 -1.91030745 -1.91030745 -1.91030745 -1.91030745  6.06488586
  3.45265902 -1.91030745  4.71662787 -1.91030745 -1.91030745 -1.91030745
 -1.91030745  5.5281319  -1.91030745 -1.91030745 -1.91030745 -1.91030745
 -1.91030745 -1.91030745 -1.91030745  3.45265902  7.8397483   6.06488586
 -1.91030745 -1.91030745]
cost : -19.527582580941242
prob : 0.72
QAOA update : [-0.00514109 -0.00902135  0.0055376   0.00401947 -0.00626    -0.00683348
 -0.00088075 -0.00638925  0.          0.          0.          0.        ]
QAOA param : [1.632002   3.32061201 2.60004404 2.84977025 1.58571944 3.11056652
 2.48322915 2.63402331 1.66846515 3.25284643 2.27502641 2.72377669]


 40%|████      | 20/50 [00:05<00:07,  3.77it/s]


KeyboardInterrupt: 

In [183]:
rl_qaoa.rqaoa_execute()

(tensor([ 25.04832627,   1.32483753,  -1.41418839,   7.19640213,
         -11.42495369,   3.94936049,  -7.98013616,  -2.11756256,
           0.        ,   0.        ,   0.        ,   0.        ], requires_grad=True),
 tensor([-2.01460178e-02, -5.86652053e-02,  1.95425309e-01,
         -1.36009914e-01, -6.19250354e-02, -8.09357575e-03,
          8.31741236e-02, -6.80286008e-05, -1.37083746e-02,
         -1.27727838e-03,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00], requires_grad=True),
 tensor(-29.28271388, requires_grad=True))

In [14]:
rl_qaoa.b

tensor([25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25.,
        25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25.,
        25., 25., 25., 25.], requires_grad=True)

In [46]:
rl_qaoa._qaoa_edge_expectations(Q,[0,1])

[tensor(-0.16702916, requires_grad=True),
 tensor(0.10821649, requires_grad=True),
 tensor(0.05888816, requires_grad=True),
 tensor(0.06899251, requires_grad=True),
 tensor(-0.16638491, requires_grad=True),
 tensor(-0.00021, requires_grad=True),
 tensor(-0.12248088, requires_grad=True),
 tensor(-0.36819676, requires_grad=True)]

In [62]:
rl_qaoa._qaoa_edge_expectations(Q,[0,1])

[tensor(-0.02168407, requires_grad=True),
 tensor(-0.07840502, requires_grad=True),
 tensor(0.07189548, requires_grad=True),
 tensor(0.14097381, requires_grad=True),
 tensor(0.01973759, requires_grad=True),
 tensor(-0.01024301, requires_grad=True),
 tensor(-0.25178965, requires_grad=True),
 tensor(-0.25886579, requires_grad=True)]

In [ ]:
rl_qaoa.

In [60]:
rl_qaoa.n_c = 4
final_config = rl_qaoa.rqaoa_execute()


In [61]:
final_config

(tensor([50.40669303,  5.81959104,  0.        ,  0.        ], requires_grad=True),
 tensor([-2.29578388e-05, -2.93378893e-04, -2.68661969e-04,
         -2.84728087e-03, -2.21101320e-05, -8.51516355e-06,
          1.86186705e-01, -1.83475578e-01,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00], requires_grad=True),
 tensor(-23, requires_grad=True))

In [59]:
init_params = [params]*2
init_params = np.reshape(init_params,(-1))
rl_qaoa.param = init_params